# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークでマルコフモデルを作成し、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab, build_vocab_from_iterator
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [2]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [3]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [4]:
data = data[:100]


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、トークン化と呼ぶ。

トークン化には、分かち書き同様形態素解析を用いる。各文章をトークンのリストとして格納する。

In [5]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
for sentence in data:
    tokens = tagger.parse(sentence).strip().split(' ') # トークンのリスト
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、トークン1つ1つにIDを割り当てる。

<br>

torchtextの`vocab`モジュールを使用する。

[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [6]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [7]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [8]:
v = build_vocab_from_iterator([
    ['私', 'は', 'りんご', 'が', '好き'],
    ['私', 'は', 'バナナ', 'を', '食べる'],
    ['今日', 'は', '晴れ', 'です']
])
v.get_itos() # vocabrary

['は', '私', 'が', 'です', 'りんご', 'を', 'バナナ', '今日', '好き', '晴れ', '食べる']

これを使ってトークン列をID列に変換する。

In [9]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

print('num of vocabrary:', len(v))
data_ids[0][:10] # example

num of vocabrary: 4602


[17, 3681, 3742, 2, 4, 3873, 4415, 47, 19, 70]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        x, y = [], []
        for ids in data_ids:
            for id1, id2 in zip(ids[:-1], ids[1:]):
                x.append(id1)
                y.append(id2)
        self.x = x
        self.y = y
        self.n_data = len(x)

    def __len__(self):
        return self.n_data

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 21128
tensor([1421,  573,    5,  853,    2,   11,  593, 1687, 1769,   11,  137,    1,
           2,   31,   74,   10,    9,  139,   36,  466,    7, 4504,   23,    2,
         617, 1074,    8,   10,    5,    4, 1564,   24,  139,   67,    0, 2665,
          58,  492,   12,    1,    8, 3535, 2280,   25,   12, 2142, 1766,  480,
           6, 3443,  446,   57, 1936,   21,    0, 3905,  716,  375,    7,  807,
         653,   57,    1,   10])
tensor([  23,  373, 3756,  477, 1750,    8,    2,  345,   41,    8,    1,  778,
         909,    6,    4, 2192,   20,   33, 1687,    0,    1, 1754,  192, 3465,
         337,   12,   18,   73, 3350,   17,    5,  474,   10,    1,  768, 1229,
          30,  800,   15,  138,    1, 3341, 3443, 3409,   15, 2153,    5,    0,
           3, 2020,  696, 1396,   48,  140,  388,   71,    2,  711,   39,    2,
           0, 1243,  292,  498])



----

## モデル構築

マルコフモデルはある状態から次の状態を予測する。言語モデルに当てはめると、ある単語から次の単語を予測する。  
これをNNで実装するので、ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDはカテゴリ変数なので、入力時はone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルとする。つまりこのタスクは単語の分類問題となる。

In [11]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.net = nn.Sequential(
            nn.Linear(vocab_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, x):
        x = F.one_hot(x, self.vocab_size).to(torch.float32)
        y = self.net(x)
        return y


---

## 学習

通常の分類モデルと同じように学習する。

In [12]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, dataloader, n_epochs):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_epoch = 0
        for x, t in tqdm(dataloader, desc=f'{epoch}/{n_epochs}', leave=False):
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        loss = loss_epoch/len(dataloader)
        print(f'{epoch}/{n_epochs} loss: {loss}', flush=True)

In [13]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
train(model, optimizer, dataloader, 20)

1/20 loss: 6.605683596112577


2/20 loss: 5.625065593200868


3/20 loss: 5.085481930355291


4/20 loss: 4.570844434899506


5/20 loss: 4.125345747276375


...

16/20 loss: 2.7113054652228454


17/20 loss: 2.6998196321671823


18/20 loss: 2.6939821855539283


19/20 loss: 2.693750683994812


20/20 loss: 2.684215051169842



---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。ある単語からの次の単語の予測を繰り返すことで文章を生成する。初めの単語だけはこちらで指定する。  
以下の条件を満たすまで単語の生成を続ける。
- 単語数が指定した限界に達する
- 読点が出力される

In [15]:
end_token_id = v.get_stoi()['。'] # 読点のID

def generate_sentence(model, start_word, max_len=30):
    model.eval()
    token_id = v.get_stoi()[start_word]
    token_ids = [token_id]

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != end_token_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        y = F.softmax(y, dim=0)

        # 分布に基づいたサンプリングを行う
        token_id = random.choices(range(len(y)), weights=y)[0]
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [16]:
start_words = ['今日', '地球', '科学', '人']
for start_word in start_words:
    print(generate_sentence(model, start_word))

今日の古賀が、末代の頃）からは、陸大教官、ガミラス艦自体2隻の取締役に留まったこととした。
地球艦隊の倍のうちにもある。
科学奴隷」プロデューサーの藤井が良いが少なかった。
人ほどで育ったブラウンシュヴァイク系金融機関でもなった。



---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### BOS

*Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### EOS

*End of Sentence*

文章の終わりを意味するトークン。  
先程の例では読点が出た場合に生成を止めたが、本来読点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けることが多い。

<br>

では特殊トークンを使って学習させてみよう。使う特殊トークン上の二つ。

In [17]:
data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

全ての文章にBOSとEOSを追加する。  
他のトークンと重複しないような文字列を指定する。

In [18]:
BOS, EOS = '<BOS>', '<EOS>'
for tokens in data_tokens:
    tokens.insert(0, BOS)
    tokens.append(EOS)

data_tokens[0][:10] # example

['<BOS>', '「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」']

残りの学習は同じ

In [19]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [20]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [21]:
train(model, optimizer, dataloader, 20)

1/20 loss: 6.619048892380949


2/20 loss: 5.617160797119141


3/20 loss: 5.078009182821491


4/20 loss: 4.562118403212039


5/20 loss: 4.125937887294564


...

16/20 loss: 2.7240183667508426


17/20 loss: 2.7129370959219106


18/20 loss: 2.70685009934945


19/20 loss: 2.698874629543213


20/20 loss: 2.694811746031938


では文章を生成する。初めのトークンはBOSとし、EOSが出力されるまで生成を続ける。

In [22]:
end_token_id = v.get_stoi()[EOS]

def generate_sentence(model, max_len=50):
    model.eval()
    token_id = v.get_stoi()[BOS]
    token_ids = []

    for _ in range(max_len):
        x = torch.tensor(token_id).unsqueeze(0).to(device)
        y = model(x)[0]
        y = F.softmax(y, dim=0)
        token_id = random.choices(range(len(y)), weights=y)[0]
        if token_id == end_token_id:
            break
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [23]:
for _ in range(5):
    print(generate_sentence(model))

1世紀初頭から再生可能であった汚職へ進出していない平日の講道館に登場する白文帳・マリアより多くのところを統括した「コスプレコンテスト、古来からチューブ付き楽曲を統括・シニアの決勝
PSO2回ほど盛んな話。
2013年）。
2018年大会芸能界入り、《恋」となっている。アルベドを仕掛けるも感謝祭2017年に行って開催する白文帳・海軍機関コードは『テイルズオブマガジン』の関心にはhydeは「コスプレ以外に
選手権大会では、ルーズリーフ、他金融機関少将にとどまる。
